# Project 3

# 뉴스 자동 분류

## 처음은 이전 Project 2와 동일하다

In [2]:
import os

def getFileList(base='./', ext='.txt'):
    fileList = list()
    
    for file in os.listdir(base):
        if file.endswith(ext): # == if file.split('.')[-1] == ext:
            fileList.append('{0}/{1}'.format(base, file))
            
    return fileList

In [3]:
# def getNewsRank():
#     for file in os.listdir(base):
#         with open(file,encoding='utf-8') as f:
#             content = f.read()
#         return content

In [4]:
def getContent(file):
    with open(file, encoding='utf-8') as f:
        content = f.read()
    
    return content

In [5]:
corpus= list()
for a in getFileList('./News'):
    document = getContent(a)
    corpus.append(document)



In [6]:
type(corpus)

list

In [7]:
len(corpus)

125

# 전처리

In [8]:
def ngramEojeol(sentence, n=2):
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [9]:
def ngramUmjeol(term, n=2):
    ngram = []
    
    for i in range(len(term) - n + 1):
        ngram.append(''.join(term[i:i + n]))   
        
    return ngram

In [10]:
def getPatternList():
    patternList = {}
    patternList['Korean'] = re.compile(r'([^ㄱ-ㅎㅏ-ㅣ가-힣]+)')
    patternList['rmeng'] = re.compile(r'[a-zA-Z]')
    patternList['Email'] = re.compile(r'(\w+@[a-zA-Z0-9\-\_]{3,}(.[a-zA-Z]{2,})+)')
    patternList['Whitespace'] = re.compile(r'\s{2,}')
    patternList['Punctuation'] =  re.compile(r'[%s]{2,}' % re.escape(punctuation))
    patternList['Punctuation2'] =  re.compile(r'[%s]' % re.escape(punctuation))
    return patternList

In [11]:
import re
from string import punctuation
corpus2 = list()

for i in range(len(corpus)):
    
    corpus[i] = getPatternList()['Email'].sub(" ", corpus[i])
    corpus[i] = getPatternList()['Whitespace'].sub(" ", corpus[i])
    corpus[i] = getPatternList()['Punctuation'].sub(" ", corpus[i])
    corpus[i] = getPatternList()['Punctuation2'].sub(" ", corpus[i])
    corpus[i] = getPatternList()['rmeng'].sub("", corpus[i])
#     corpus[i] = getPatternList()['Nonword'].sub(" ", corpus[i])
#     corpus[i] = getPatternList()['Numeric'].sub(" ", corpus[i])
#     corpus[i] = getPatternList()['Maxlength'].sub(" ", corpus[i])
    corpus2.append(corpus[i])

In [12]:
from nltk.tokenize import sent_tokenize

newCorpus = list()
for i in range(len(corpus)):
    tokenNews = sent_tokenize(corpus[i]) 
    newCorpus.extend(tokenNews)

In [13]:
# sent_tokenize한 corpus
# newCorpus

In [14]:
from konlpy.tag import Kkma

dictTerm = list()
dictPos = list()
dictNoun = list()
dictNgram = list()

for sentence in newCorpus:
    for token in sentence.split():
        if len(token) > 1:
            dictTerm.append(token)
            dictPos.extend([morpheme for morpheme in Kkma().morphs(token) if len(morpheme) > 1])
            dictNoun.extend([noun for noun in Kkma().nouns(token) if len(noun) > 1])
            dictNgram.extend(ngramUmjeol(token))
        
dictTerm = list(set(dictTerm))
dictPos = list(set(dictPos))
dictNoun = list(set(dictNoun))
dictNgram = list(set(dictNgram))

In [15]:
len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram)

(18933, 7084, 8797, 15771)

In [657]:
# print(dictTerm)

In [658]:
# print(dictPos)

In [659]:
# print(dictNoun)

In [660]:
# print(dictNgram)

In [20]:
def getNewsBySet():
    corpus3 = list()
    for docName in getFileList('./News'):
        document = getContent(docName)
        
        for token in document.split():
            corpus3.append(token)

    return list(set(corpus3))

In [661]:
txt = getNewsBySet()
# txt

In [22]:
from konlpy.tag import Kkma
from collections import defaultdict

ma = Kkma().morphs
def getNewsReprByDefaultDict(document):
    docRepr = defaultdict(lambda: defaultdict(int))
    
    for docName in getFileList('./News'):
        document = getContent(docName)
        
        for token in document.split():
            for morpheme in ma(token):
                docRepr[docName][morpheme] += 1
    return docRepr

In [662]:
DTM = getNewsReprByDefaultDict(txt)
# DTM

# 가중치

In [24]:
from math import log10

def rawTF(freq):
    return freq

def normTF(freq,totalCount):
    return (freq / totalCount)

def logTF(freq):
    if freq > 0:
        return 1 + log10(freq)
    else:
        return 0

def maxTF(a,freq,maxFreq):   # double normalization K -  doc : 0 / query : 0.5
    return a + ((1-a)* (freq/maxFreq))

In [25]:
def convertInvertedDocument(DTM):
    TDM = defaultdict(lambda: defaultdict(int))
    
    for fileName, termList in DTM.items():  
        maxFreq = max(termList.values())
        for term, freq in termList.items():
            TDM[term][fileName] = maxTF(0,freq,maxFreq)
            
    return TDM

In [663]:
TDM = convertInvertedDocument(DTM)
# TDM

In [27]:
# 일반적인 IDF
def rawIdf(df, N):
    return log10(N / df)

# the,a, 불용어 안날림 => to be or not to be 
def smoothingIdf(df,N):
    return log10((N+1) / df)

def probabilityIdf(df,N):
    return log10((N-df+1) / df)

In [28]:
# term-document -> term weight
# defaultdict 를 써서 key 걱정을 안해도 된다. 
N = len(DTM)

def TDM2TWM(TDM):
    TWM = defaultdict(lambda: defaultdict(float))
    DVL = defaultdict(float)
    
    for term, tfList in TDM.items():
        df = len(tfList)
        idf = rawIdf(df,N)
        for fileName, tf in tfList.items():
            TWM[term][fileName] = tf * idf
            DVL[fileName] += TWM[term][fileName]  ** 2
            
    return TWM, DVL
    

In [29]:
TWM,DVL = TDM2TWM(TDM)

In [664]:
# TWM

In [665]:
globalTF = list()
globalDocument = list()


for (docName, docContent) in enumerate(newCorpus):
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    localPosting = dict()
    maxCount = 0
    # 로컬 / 띄어쓰기 단위로 
    for term in docContent.lower().split():   
        maxCount += 1
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
            
    print(docName)
    a = 0.5
    maxFreq = max(localPosting.values())
    
    for term,freq in localPosting.items():
#         print("1. {0} rawTF : {1}".format(term,rawTF(freq)))
#         print("2. {0} normTF : {1}".format(term,normTF(freq,maxCount)))
#         print("3. {0} logTF : {1}".format(term,logTF(freq)))
#         print("4. {0} maxTF : {1}".format(term,maxTF(a,freq,maxFreq)))
#         print()
        localPosting[term] = maxTF(a,freq,maxFreq)
        
    for indexTerm, termTF in localPosting.items():
        if indexTerm not in localPosting.keys():
            lexiconIdx = len(localPosting)
            postingIdx = len(globalTF) # fseek
            postingData = (lexiconIdx,docIdx,termTF, -1)
            globalTF.append(postingData)
            localPosting[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
        else:
            lexiconIdx = list(localPosting.keys()).index(indexTerm)
            postingIdx =  len(globalTF)
            beforeIdx = localPosting[indexTerm]
            postingData = (lexiconIdx,docIdx,termTF, beforeIdx)
            globalTF.append(postingData)
            localPosting[indexTerm] =  postingIdx           

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [32]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in enumerate(newCorpus):
    # Pointer 대체용, Key, Document이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # 로컬 / 띄어쓰기 단위로 
    for term in docContent.lower().split():   
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    # fp -> struct(단어,빈도) ( localPosting)
    # Merge와 sorting이 같이 있는 것 
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = [lexiconIdx,docIdx,maxTF(0,termFreq,maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx =  len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx,docIdx,maxTF(0,termFreq,maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] =  postingIdx   # globalPosting 위치 (ptr:idx)
#     print(localPosting)
# print(globalDocument)  
#         if term not in globalLexicon.keys():
#             localPosting
#             lexiconIdx = len(globalLexicon)   # 처음엔 Length가 0일 것. 
            
            

In [33]:
query = "서울시에 거래되는 아파트 전세값은?"

In [34]:
queryRepr = defaultdict(int) # 빈도를 갖는

for token in query.split(): 
    for morpheme in ma(token):
        queryRepr[morpheme] += 1
        
queryWeight = defaultdict(float)
maxFreq = max(queryRepr.values())

for token, freq in queryRepr.items():
    if token in TWM.keys():
        tf = maxTF(0.5,freq,maxFreq)
        df = len(TWM[token])
        idf = rawIdf(df,N)
        queryWeight[token] = tf * idf

In [35]:
queryWeight

defaultdict(float,
            {'서울시': 1.494850021680094,
             '에': 0.0,
             '거래': 1.193820026016113,
             '되': 0.01055018233330815,
             '는': 0.0,
             '아파트': 1.6197887582883939,
             '값': 1.7958800173440752,
             '은': 0.0,
             '?': 0.453457336521869})

In [48]:
def innerProduct(x,y):
    return x * y

In [666]:
from math import sqrt

candidateList = defaultdict(float)

for token, weight in queryWeight.items():
    for fileName, tfidf in TWM[token].items():
#         print(" {0} : {1} = {2} * {3}".format(
        token, fileName,weight,tfidf))
        candidateList[fileName] += innerProduct(weight, tfidf)
        
for fileName, sumProduct in candidateList.items():
    candidateList[fileName] /= sqrt(DVL[fileName])

SyntaxError: invalid syntax (<ipython-input-666-127d9b91b91a>, line 8)

In [37]:
from nltk.tokenize import sent_tokenize
K=5

resultList = sorted(candidateList.items(), key = lambda x:x[1], reverse=True)

for i,(fileName, similarity) in enumerate(resultList):
    if i < K:
        print(" Rank:{0} / Document:{1} / Similarity:{2:.4f}".format((i+1),fileName,similarity))
        
        with open(fileName,encoding='utf-8') as f:
            content = f.read()
            content = sent_tokenize(content)
            
        print(content[:5])     

 Rank:1 / Document:./News/세계_0000718407.txt / Similarity:0.4094
['\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n『알쓸中잡』알고 보면 쓸모 있을 수 있는 중국 잡학사전138만 위안.', '최근 중국 온라인 경매에서 낙찰된 베이징 아파트 주차장 한 면의 가격입니다.', '원화로 하면 2억 3천만 원이 넘는 돈입니다.', '중국신문망에 따르면 138만 위안이면 중국 각 성의 수도에서 100 평방미터 남짓한 집 한 채를 살 수 있고, 후선 도시에서는 2채도 사는 것이 가능한 금액입니다.먼저, 중국 아파트의 특징을 말씀드려야 할 것 같습니다.', '중국의 많은 아파트들은 집값에 주차장이 포함돼 있지 않습니다.']
 Rank:2 / Document:./News/경제_0002892918.txt / Similarity:0.1386
['\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n[현장에서]1339만 가구 공동주택 공시가격550명이 4개월여간 조사·산정정부 "세부 산정 과정 공개 못한다"국민들 의혹·불신 더욱 키워  지난 14일 서울시의 아파트 단지들 [연합뉴스]           한 사람이 하루에 약 180가구의 공시가격을 계산했다.', '최근 발표된 공동주택 공시가격 이야기다.', '한국감정원 직원 550명이 지난해 8월 27일부터 올해 1월 11일까지 138일 동안 1339만 가구를 조사했는데, 이를 단순화해 본 것이다.', '조사 기간에서 쉬는 날을 빼면 1명이 하루에 맡아야 할 집 수는 180가구 이상이다.', '여기다 조사 인력 상당수는 비슷한 시기 단독주택 22만 가구의 공시가격 산정 업무도 떠안았다.']
 Rank:3 / Document:./News/세계_0003891624.txt / Similarity:0.1146
['\n\n

# KNN

1. k개의 가장 유사한 문서들의 카테고리 추출

2. 카테고리 별 투표를 통해 가장 유사한 카테고리 선정


# 카테고리별 뉴스 Query

In [352]:
# 카테고리 : "세계"

query= "손님을 사정없이 ‘밟아’ 회복불능으로 다치게 한 태국 마사지사가 벌금형을 선고받았다. 2일 말레이시아 매체 월드오브버즈는 지난해 12월 태국 유명 마사지사에게 마사지를 받고 장애를 얻은 프라놈 티엥트롱(46·여성)의 사연을 소개했다. \
태국 동남부 라용에 있는 한 마트에서 식품 판매원으로 일하는 티엥트롱은 몇 달간 계속된 다리와 엉덩이 통증으로 ‘해피 박사’라 불리는 마사지사 키아티삭 차이위몬(21)을 찾았다. 차이위몬은 족도법(발로 밟는 마사지)으로 수천 명을 치료하는 등 인근에서 꽤 알아주는 마사지사였다. \
차이위몬은 티엥트롱을 매트 위에 엎드리게 한 뒤 아픈 다리와 엉덩이를 살폈다. 엉덩이 관절이 탈구된 것이라고 진단한 그는 이내 뼈를 맞추기 위해 티엥트롱의 다리를 밟기 시작했다. \
티엥트롱은 거구의 남성이 밟는 무게를 견디기 힘들었지만, 모든 게 ‘치료 과정’이라고 생각하고 꾹 참았다. \
그러나 문제는 차이위몬이 그의 대퇴부(엉덩관절과 무릎관절을 잇는 넓적다리)를 밟았을 때 발생했다. 참을 수 없는 고통에 비명을 지른 티엥트롱은 '너무 아프다. 그만하라'고 말했다. 그럼에도 차이위몬은 '이쪽이 탈구돼서 그렇다. 뼈를 맞추고 있으니 조금만 참아라'고 다독이며 계속해서 마사지를 이어갔다. \
1시간여의 마사지를 마치고 집으로 돌아온 티엥트롱은 여전히 통증을 호소했다. 점점 나아질 거란 기대로 하루, 이틀 그리고 일주일을 버텼지만 오히려 상태가 더 나빠지고 있었다. 다른 사람의 도움 없이 혼자 걸을 수도 없는 지경이었다.\
이대로는 안 되겠다 싶어 이웃과 함께 병원을 찾은 그는 대퇴부가 골절되고 발목 관절이 탈구됐다는 진단을 받았다. 치료하기 위해서는 거금 13만 바트(약 470만 원)가 필요했다. 완치가 될지 여부는 불투명한 상황이다. 그는 앞으로 휠체어를 타거나 목발을 짚고 다녀야만 할 수도 있다.\
다리가 아파 한 달 가까이 일을 하지 못한 티엥트롱의 수중에는 그런 큰돈이 없었다. 남편과도 이혼해 그를 돌봐줄 사람 또한 없었다.\
티엥트롱은 이 모든 사달을 낸 차이위몬에게 책임을 묻기 위해 연락을 취했으나 묵묵부답이었다. 이후로도 계속 연락이 닿지 않자 결국 그를 경찰에 신고했다. 차이위몬은 자택에서 현장에 출동한 경찰에 의해 체포됐다.\
그는 경찰에 '티엥트롱의 뼈를 맞추기 위해 반복적으로 밟았다'며 '고통스러워했지만 뼈가 부러진 줄은 정말 몰랐다. 너무 세게 밟은 것 같다. 잘못했다'고 선처를 호소했다.\
경찰조사 결과 차이위몬은 마사지자격증이 없는 것으로 드러났다. 불법 시술 행위 및 의료시설 개설 등 혐의로 재판에 넘겨진 그는 벌금형을 선고받았다."


In [344]:
# 카테고리 : IT과학

query = "간편결제 이용건수·금액도 급증 \
휴대폰 송금[연합뉴스TV 캡처] \
(서울=연합뉴스) 정수연 기자 = 토스, 네이버페이, 카카오페이 등 스마트폰을 통한 간편송금 서비스의 하루평균 이용금액이 1년 만에 3배 가까이로 늘었다.\
한국은행이 3일 발표한 '2018년중 전자지급서비스 이용 현황'을 보면 지난해 간편송금 서비스 이용금액(이하 하루평균)은 1천45억원으로 전년보다 194.1% 증가했다.\
이용 건수는 141만건으로 102.5% 늘었다.\
※ 자료 : 한국은행\
간편송금은 스마트폰에 충전한 선불금을 전화번호나 소셜네트워크서비스(SNS) 등을 활용해 송금하는 서비스를 말한다.\
한은 관계자는 '지난해 토스와 카카오페이가 크게 성장하면서 간편송금 이용금액과 건수가 늘었다'고 설명했다.\
공인인증서나 OTP 없이 비밀번호 등을 이용해 결제하는 간편결제 서비스 규모도 두 배 가까이 성장했다. \
지난해 간편결제 이용금액은 하루평균 1천260억원으로 전년보다 86.2% 늘었다.\
이용 건수도 87.5% 증가한 392만건으로 조사됐다.\
[그래픽] '○○페이'로 송금한 돈 3배로 증가\
간편결제·송금 서비스를 합한 이용금액은 하루평균 2천306억원, 건수는 533만건이다.\
카카오페이, 네이버페이 등 정보·통신(ICT) 분야와 스마일페이, SSG 페이 등 유통·제조 분야 간편결제 서비스도 성장세를 이어갔다.\
온라인쇼핑이 늘어난 영향으로 전자지급결제대행(PG) 서비스 이용금액이 하루평균 4천123억원으로 25.8% 불어났다.\
이용 건수는 29.8% 증가한 826만건으로 집계됐다.\
PG는 전자상거래에서 구매자에게 받은 대금이 판매자에게 지급될 수 있도록 지급결제 정보를 송수신하는 서비스다. 온라인쇼핑이 늘어나면 자연스레 PG 서비스 이용 규모도 확대한다. \
토스 및 선불교통카드 등이 포함된 선불전자지급서비스 규모는 1천428억원, 1천635만건으로 각각 116.4%, 9.1% 늘었다.\
한은 관계자는 '간편송금을 중심으로 증가세가 확대해 전체 선불전자지급서비스 이용 규모가 늘어났다'고 설명했다.\
오픈마켓이 포함된 결제대금 예치 이용금액은 697억원, 건수는 146만건으로 각각 15.1%, 15.3% 증가했다.\
아파트 관리비를 이메일 등 전자 방식으로 발행한 다음 대금을 직접 수수하고 정산을 대행하는 서비스인 전자고지 결제서비스도 늘었다.\
전자고지 결제서비스 이용금액은 29.9% 늘어난 278억원, 건수는 26.6% 증가한 16만건으로 나타났다.\
직불전자지급 서비스 이용금액은 140.2% 증가한 1천564만원으로 나타났다. 이용 건수는 81.5% 늘어난 698건으로 조사됐다.\
직불전자지급 서비스란 스마트폰 인증 절차를 거쳐 편의점 등을 이용할 때 이미 등록된 구매자 계좌에서 판매자 계좌로 자금 이체를 중계하는 서비스를 말한다."

In [358]:
# 카테고리 : 정치
query = "'긴장 상존' 서부·중부전선 GP까지 개방…'해빙' 흐름 유지노력 일환인듯\
'대북통보 및 동의·유엔사 승인 절차 없이 서둘러 정책 발표' 지적도\
北 '9·19 군사합의' 이행 소극적인 상황서 추진하는 점도 논란 소지 \
둘레길로 개방되는 DMZ(파주=연합뉴스) 임병식 기자 = 정부는 3일 행정안전부, 문화체육관광부, 통일부, 국방부, 환경부 등 5개 부처 합동 브리핑을 열고 비무장지대(DMZ)와 연결된 3개 지역(경기 파주·강원 철원·강원 고성)을 'DMZ 평화둘레길'로 조성한다고 발표했다. 이 가운데 파주는 임진각에서 시작해 도라전망대를 거쳐 9·19 남북군사합의에 따라 철거한 GP 현장까지 방문하는 구간이다. 사진은 3일 도라전망대에서 외국인 관광객들이 기념촬영하는 모습. andphotodo@yna.co.kr\
(서울=연합뉴스) 김호준 기자 = 정부가 3일 발표한 '비무장지대(DMZ) 평화둘레길'은 민간인 관광객이 남북간 군사적 긴장이 상존하는 DMZ에 진입해 시범철수 GP(감시초소) 혹은 비상주 GP를 방문하는 관광코스다.\
관광 프로그램은 군사분계선(MDL) 이남에서만 이뤄지는 것이긴 하다. 그러나 '9·19 군사합의'에 따라 남북이 DMZ 내 GP 11개씩을 시범철수하고도 여전히 북측 150여개, 남측 50여개의 GP에서 중무장한 장병들이 경계를 서고 있어 관광객의 DMZ 진입에 따른 안전 문제가 숙제로 제기되고 있다. \
아울러 관광객들이 전방지역의 민간인통제선을 넘어 DMZ까지 들어섬에 따라 국민의 생명과 안전을 책임져야 할 군의 경계부담이 커질 수 있다는 우려도 나온다. \
정부는 이날 ▲경기도 파주에 있는 임진각에서 출발해 도라전망대를 경유해 시범철수 파주GP까지 방문하는 서부전선 코스 ▲강원도 철원 백마고지 전적비에서 출발해 DMZ 내 화살머리고지까지 방문하는 중부전선 코스 ▲통일전망대에서 출발해 금강산전망대를 견학하고 복귀하는 동부전선 코스 등 'DMZ 평화둘레길'을 단계적으로 민간에 개방한다고 밝혔다. \
서부전선과 중부전선은 DMZ 안으로 진입하는 코스이고, 동부전선은 DMZ 남방한계선(철책) 부근까지만 접근하는 관광코스다.\
정부는 이달 말부터 GOP(일반전초) 철책선 이남의 고성지역(동부전선)을 대상으로 시범운영을 시작하고, 파주(서부전선)와 철원(중부전선) 지역 둘레길도 방문객 접수를 위한 준비가 마무리 되는대로 이어서 개방할 예정이다. \
당초 DMZ 안으로 진입하는 서부전선과 중부전선 코스도 이달부터 개방할 예정이었으나 관광객 안전 문제 등을 보완해 시행키로 했다. 서부전선과 중부전선 둘레길 개방시기는 5~6월로 예상된다. \
국방부는 '군(軍)은 확고한 군사대비 태세를 갖춘 가운데 경계작전에 지장이 없는 범위 내에서 방문객의 견학이 가능토록 조치했다'며 '개방되는 GP는 철거된 GP 또는 비상주 GP로, 현행 작전에 영향을 주지 않는 지역'이라고 설명했다. \
국방부는 또한 '경호지원도 군단 특공연대에서 제공해 현행 작전부대의 부담을 최소화했다'며 '특히, DMZ 내 이동 때는 군 경호 아래 차량으로 단체 이동하는 등 철저한 안전관리 대책을 마련하고 있다'고 강조했다. \
문 대통령, 화살머리 GP 방문(철원=연합뉴스) 배재만 기자 = 문재인 대통령이 28일 강원도 철원 화살머리 GP를 방문, 동행한 장병들을 격려하고 있다. 2018.12.28 scoop@yna.co.kr\
관광객이 3중 철책이 설치된 DMZ 남방한계선을 통과해 GP까지 이동할 때는 차량으로 이동하지만, 철거 GP와 비상주 GP에 도착해서는 차량에서 내려 북측 지역을 조망하게 된다. \
문제는 GP에 도착해 하차했을 때 여전히 운영되는 북측 GP 중화기에 노출된다는 점이다. 관광객이 방문하게 될 파주 철거 GP와 지금도 운영되는 가장 가까운 북측 GP와의 거리는 1.5㎞ 이내로 북측 GP에 배치된 고사총의 사거리 안에 들어온다. \
아울러 DMZ 남북 장병이 지금도 수색·매복 작전을 수행하는 지역이기 때문에 관광객이 DMZ 남측지역을 차량으로 이동하는 중에도 군사적 위협에 노출될 가능성을 완전히 배제할 수 없다. \
게다가 GP 혹은 GOP(일반전초)에서 근무하는 국군 장병들도 관광객의 유출입에 따라 북한군의 동향에 촉각을 곤두세워야 하므로 경계부담이 늘어날 수밖에 없다. 어떤 상황이 벌어질지 장담할 수 없기 때문에 매일 전방의 북한군 동향을 면밀히 살펴야 한다. \
국방부는 이번 DMZ 평화둘레길은 지난해 체결된 9·19 군사합의 이후 남북 간에 쌓아온 신뢰를 전제로 한 사업이라고 설명하나 올해 들어 군사합의는 북한의 소극적인 태도로 원활히 이행되지 않고 있다. \
남북이 군사합의를 통해 4월 1일부터 화살머리고지에서 시작하기로 약속한 DMZ 남북공동유해발굴도 북한이 호응하지 않아 남측 단독으로 해당 지역에서 기초 유해발굴 작업을 진행 중이다. \
DMZ 남북공동유해발굴 북측 무응답 (PG)[정연주 제작] 일러스트\
우리 정부가 DMZ 평화둘레길 사업을 발표하기 전에 북측에 통보해 동의를 얻는 절차가 없었고, 정전협정에 따라 DMZ를 관할하는 유엔군사령부의 승인을 받지 않은 것도 문제로 지적된다. \
국방부 당국자는 '우리 지역에서 안보 견학을 하는 것이기 때문에 북한에 통보할 의무가 있는 사항은 아니다'면서 ''(북측 통보) 필요성도 검토하겠다'고 말했다. \
군 당국은 DMZ로 진입하는 서부전선과 중부전선 코스는 유엔사와 협의를 마치고 민간에 개방할 방침인 것으로 전해졌다. \
이 당국자는 유엔사 승인 문제에 대해서는 '유엔사와 협의를 진행 중'이라며 '유엔사도 안전에 문제가 없다는 것만 확인되면 승인절차에는 문제가 없다는 게 기본 입장'이라고 전했다. \
물론 국방부도 민간인 DMZ 관광의 안전을 100% 장담할 수는 없다는 입장이다. 이 때문에 군 당국은 DMZ 평화둘레길의 코스와 관광 방식을 결정하는 과정에서 안전 우려를 최소화할 것을 정부에 건의한 것으로 알려졌다. \
국방부 관계자는 ''(DMZ 평화둘레길 관광이) 100% 안전하다고 군인 입장에서는 말씀드릴 수는 없지만, GP 후방지역(DMZ 남측지역)이기 때문에 안전이 확보된 지역이라고 생각하면 된다'고 말했다. \
이 관계자는 안전장비와 관련해서는 '민수용 방탄복과 방탄 헬멧을 경호차량에 휴대하고 가게 된다'고 설명했다. \
도라전망대에서 바라본 개성공단(파주=연합뉴스) 임병식 기자 = 3일 경기도 파주시 서부전선 비무장지대(DMZ) 도라전망대에서 바라본 개성공단의 모습. andphotodo@yna.co.kr\
군 당국은 관광객이 GP에 도착해 차량에서 내릴 때는 방탄복과 헬멧을 지급할 방침인 것으로 전해졌다. \
이 프로그램은 단순한 관광 프로젝트를 넘어 2월 북미정상회담 결렬 이후 한반도 정세의 불확실성이 커진 상황에서 한국 주도로 한반도 긴장 완화의 분위기를 이어가는 노력의 일환으로 보이는 측면이 있다. 아울러 향후 북미협상의 방향을 고민하는 북미 양측과 국제사회를 향해 한반도 평화의 흐름에 지속 동참하기를 촉구하는 메시지를 전하는 목적이 내포됐을 수 있다는 평가도 나온다.\
국방부는 '남북 분단의 상징이었던 GOP 이북 DMZ를 일반 국민에게 최초로 개방하는 조치'라며 '한반도 평화정착을 향한 국민적 공감대가 확산하기를 기대한다'고 DMZ 평화둘레길 사업 추진 배경을 설명했다. \
둘레길로 개방되는 비무장지대(파주=연합뉴스) 임병식 기자 = 정부는 3일 행정안전부, 문화체육관광부, 통일부, 국방부, 환경부 등 5개 부처 합동 브리핑을 열고 비무장지대(DMZ)와 연결된 3개 지역(경기 파주·강원 철원·강원 고성)을 'DMZ 평화둘레길'로 조성한다고 발표했다. 이 가운데 파주는 임진각에서 시작해 도라전망대를 거쳐 9·19 남북군사합의에 따라 철거한 GP 현장까지 방문하는 구간이다. 사진은 지난 2월 촬영한 장단면사무소의 모습이며 둘레길을 따라 철거한 GP로 이동하면서 이 건물을 볼 수 있다. 1934년에 지어진 장단면사무소는 일제시대 때 장단 지역의 행정 업무를 맡았으며 현재 한국전쟁 당시 총탄 흔적을 가진 채 DMZ 안에 남아 있다. andphotodo@yna.co.kr\
그럼에도 군 일각에선 DMZ 내 GP가 모두 철거되는 수준으로 9·19 군사합의가 이행되는 단계에서 시작해도 늦지 않을 텐데 정부가 너무 서둘러 DMZ 평화둘레길 관광을 시작하려 한다는 의견도 제기된다. \
지난 2월 말 2차 북미정상회담 결렬 이후 한반도 비핵화와 평화정착을 목표로 한 남북 및 북미 대화도 교착상태인 터에 굳이 DMZ 평화둘레길 관광을 현시점에서 추진할 필요가 있느냐는 지적도 있다.\
익명을 요구한 군의 한 관계자는 '2008년 7월 금강산 관광객 고(故) 박왕자씨가 북한군 초병의 총격으로 사망한 사건을 교훈 삼아 DMZ 평화둘레길 관광을 신중하게 추진할 필요가 있다고 말했다."

In [364]:
# 카테고리 : 경제 
query = "미중 무역분쟁에 세계 경기 둔화 영향\
전문가들 '올해 코스피 이익 정체 또는 감소 전망…하반기 회복 기대'\
작년 상장사 외형 커졌지만 이익은 부진…순이익 112조원 (CG)[연합뉴스TV 제공]\
(서울=연합뉴스) 임미나 기자 = 지난해 코스피와 코스닥 상장사들이 매출은 늘었지만, 이익은 부진했던 것으로 나타났다.\
재작년에 상장사들이 역대 최대 실적을 달성한 데 이어 지난해에도 상승세를 이어갈 것으로 당초 기대됐으나, 반도체 부문 실적이 작년 4분기부터 급감하면서 영업이익은 제자리 수준에 머물렀고 순이익은 소폭 줄었다. \
증시 전문가들은 세계 경기 둔화 흐름 속에 반도체 수출이 올해 상반기까지 부진할 전망이어서 올해 국내 기업의 이익이 작년과 비슷하거나 더 줄어들 것으로 예상했다.\
다만 상반기 기업들의 이익 수준이 바닥을 통과해 하반기에 회복세를 보일 가능성이 있다고 진단했다.\
◇ 경기 둔화·반도체 업황 4분기 추락에 이익 감소\
코스피 상장사는 매출과 영업이익이 작년보다 소폭 늘었지만 순이익은 줄었다.\
3일 한국거래소와 한국상장회사협의회가 12월 결산 코스피 상장사 540개사(금융업 제외)의 연결 실적을 분석한 결과 지난해 매출은 1천894조7천억원으로 전년보다 4.76% 증가했다.\
영업이익도 157조7천억원으로 0.32% 늘었으나, 당기순이익은 108조원으로 6.72% 줄었다.\
코스닥 상장사는 매출만 늘고 영업이익과 순이익은 줄었다.\
거래소와 코스닥협회가 분석한 911개 코스닥 기업의 작년 연결기준 매출은 169조1천억원으로 전년보다 4.69% 늘었다.\
그러나 영업이익과 순이익은 8조4천억원과 4조3천억원으로 각각 11.58%, 8.66% 감소했다.\
코스피와 코스닥 기업을 합친 상장사 순이익은 112조3천억원으로 전년의 120조5천억원보다 6.8% 줄었다.\
조용준 하나금융투자 리서치센터장은 '지난해 세계 경기 성장세가 둔화했고 미국과 중국의 무역분쟁으로 교역량이 급감하면서 국내 수출기업들의 실적이 부진했다'며 '특히 4분기부터 반도체 실적이 크게 하락한 탓이 크다'고 분석했다.\
이창목 NH투자증권 리서치센터장도 '재작년에 코스피 영업이익이 사상 최대를 기록하면서 작년에도 그 이상이 나올 거라고 당초 예상했으나 결과는 좋지 않았다'며 '반도체 부문 이익이 4분기로 갈수록 점점 줄어든 것이 큰 영향을 줬다'고 말했다. \
기업들의 이익지표도 전년보다 나빠졌다.\
코스피 상장사들의 매출액 영업이익률은 8.32%로 전년보다 0.37%포인트, 매출액 순이익률은 5.70%로 0.70%포인트 각각 하락했다.\
코스닥 상장사의 경우에도 영업이익률이 4.98%, 순이익률이 2.55%로 각각 0.92%포인트, 0.38%포인트 낮아졌다.\
정보기술(IT)·반도체 등 일부 업종에 이익 성장세가 쏠리는 현상도 여전했다.\
코스피 상장사 전체 매출 중 12.87%를 차지하는 삼성전자를 제외할 경우 전체 매출액은 1천650조원으로 전년보다 5.22% 증가했으나, 영업이익은 98조8천억원으로 4.57% 줄어 마이너스로 반전했다.\
삼성전자를 제외한 순이익도 63조6천억원으로 13.51% 줄어 삼성전자 포함 시보다 감소율이 약 2배로 심해졌다.\
코스닥 시장에서는 정보기술(IT)업종이 상대적으로 호조를 보였다.\
연결기준으로 IT업종 344개사의 작년 매출액은 4.21% 늘었고 순이익은 18.0% 증가했다.\
반면 나머지 업종 567개사의 매출액은 4.95% 증가했으나, 순이익은 22.52% 감소했다.\
◇ 올해도 코스피·코스닥 이익 정체될 전망\
증시 전문가들은 대체로 올해도 상장사의 실적이 좋지 않을 것으로 봤다. \
미중 무역협상이 당장 타결될 조짐이 보이지 않고 반도체 업황도 상반기까지 침체를 지속해 수출이 회복되기 어렵다는 진단이다.\
오현석 삼성증권 리서치센터장은 '미중 무역분쟁 여파가 상반기까지 반영될 것'이라며 '반도체 이익이 반 토막 이상 급감할 가능성이 커 연간으로 보면 코스피 이익은 작년보다 감소한다고 봐야 할 것'이라고 말했다.\
이창목 센터장도 '올해는 코스피 이익이 작년보다 더 감소할 가능성이 있다'며 '반도체 등 수출량이 감소할 것이 확실해 실적 부진에 영향을 줄 것'이라고 내다봤다.\
고태봉 하이투자증권 리서치센터장은 '올해 코스피 영업이익은 작년보다 15%가량 감소할 것으로 예상한다'며 '세계 경기 하강과 수출주 중심 경기민감주의 부진 등의 영향이 클 것'이라고 분석했다.\
코스피 이익이 하반기부터 회복 추세로 돌아서서 연간 이익은 작년과 비슷한 수준을 보일 것이란 전망도 있다.\
조용준 센터장은 '세계 경기가 연착륙할 가능성이 커지고 각국의 경기부양책도 서서히 효과가 나타나면서 하반기에는 기업들의 실적이 반등할 가능성이 있다'고 전망했다.\
조 센터장은 '반도체 업황도 하반기부터 회복되면서 올해 연간으로 보면 기업 이익이 작년과 비슷하게 유지되거나 소폭 감소할 것으로 보인다'고 덧붙였다.\
코스닥 상장사들도 대기업과 연계된 부품업체들의 이익 감소가 불가피해 실적이 부진할 것으로 관측됐다.\
이창목 센터장은 '코스닥 역시 대기업과 연결된 업체들이 많아서 1∼2분기에 실적이 좋지 않을 것으로 본다'며 '다만 코스닥 주가는 당장의 실적보다 미래에 대한 전망이 크게 영향을 주는 만큼 올해 실적 부진이 증시에 미치는 영향은 제한적일 것'이라고 말했다."

In [331]:
# 카테고리 : 사회 
query = "인상착의 비슷한 '여장남자' 목격담…경찰 미제팀 ‘전력 수사’\
'포천 여중생 살인사건' 용의자(?)(포천=연합뉴스) 권숙희 기자 = 2004년 발생한 '포천 여중생 살인사건' 관련, 당시 유사한 사건의 피해를 입을 뻔했다고 주장하는 여성의 기억을 토대로 제작된 몽타주. [경기북부지방경찰청 제공]\
(의정부=연합뉴스) 권숙희 기자 = 경기도 포천시에서 90여일간 실종됐다가 싸늘한 주검으로 발견된 여중생 엄모양 살인사건의 실마리가 15년 만에 풀릴까.\
지난달 30일 SBS TV프로그램 '그것이 알고싶다'에서 이 사건을 방영, 미제 살인사건의 용의자일 가능성이 있는 한 남성의 몽타주를 공개하면서 세간의 관심이 쏠리고 있다.\
3일 경기북부지방경찰청 미제사건전담수사팀에 따르면 최근 사흘간 몽타주와 인상착의가 비슷한 사람을 봤다는 112신고 전화 등이 총 10건 접수됐다.\
내용은 주로 '몽타주와 인상착의가 비슷한 여장남자를 목격했다'는 등 서로 비슷했다.\
'포천 여중생 살인사건'은 2004년 2월 8일 경기도 포천시 소흘읍의 한 배수로에서 실종된 지 3개월이 지난 엄모(당시 15세)양이 얼굴에서 가슴까지 훼손돼 형체를 알아보기 어려운 상태로 숨진 채 발견된 사건이다.\
엄양의 손톱과 발톱에 붉은색 매니큐어가 칠해진 것이 특징으로, '포천 매니큐어 살인사건'으로도 불렸다.\
당시 경찰 수사본부는 1년간이나 대대적인 수사를 벌였으나 현장 주변의 폐쇄회로(CC)TV는 물론, 다른 단서나 제보도 없어 결국 미제 사건으로 남게 됐다.\
그러던 중 15년 만에 자신이 유력한 목격자일 수도 있다고 주장하는 시민이 나타났다.\
지난달 7일 새벽, 사건 당시 대학생이던 한 여성이 112에 직접 신고 전화를 했다.\
경찰은 다음날 바로 이 여성과 면담을 진행했으며, 단서 수집에 나섰다. \
이 여성은 경찰에 엄모양이 실종되던 시기 자신도 '흰색 차량'에 의해 납치될 뻔했으며, 그 사건을 겪은 후 얼마 지나지 않아 동네에 엄모양 실종 관련 현수막이 내걸린 것을 봤다고 진술했다.\
이 여성은 당시 차에서 가까스로 탈출해 위기를 모면했으며, 15년이 지나 스마트폰으로 인터넷을 하다가 우연히 '포천 여중생 살인사건' 관련 게시물을 접한 뒤 급하게 제보를 하게 됐다고 덧붙였다.\
몽타주는 이 여성의 기억을 토대로 그려졌다.\
'포천 여중생 살인사건' 현장감식[연합뉴스 자료사진]\
방송 이후 쏟아진 신고 전화 중에 아직 결정적인 제보는 접수되지 않았으며, 전반적인 검토 작업을 하고 있다. \
또 해당 남성이 유력 용의자가 아닐 가능성도 배제할 수 없는 상황에서, 인권 침해의 여지 등을 고려할 때 목격담만으로 전부 용의선상에 올릴 수도 없기 때문이다.\
한편, 이 사건을 포함해 총 13건의 미제사건을 전담하고 있는 경기북부 수사팀의 현재 인원도 4명밖에 없어, 서울(11명), 경기남부(7명), 부산(6명) 등과 비교하면 부족한 편이다. \
2004년 이 사건이 전국적인 관심을 받을 당시 범인 검거에 부담을 느낀 포천경찰서 강력1반장 윤모(47) 경사가 스스로 목숨을 끊는 일도 있었다."


In [376]:
# 카테고리 : 생활문화
query = "일본 도쿄(東京)에는 동물을 보고 만질 수 있는 동물카페가 많다. 우리나라에는 방송에서 연예인들이 동물카페를 방문한 장면들이 나오면서 알려졌다. 일본 관광책자에도 동물카페 소개를 찾아보긴 어렵지 않다. 도쿄 동물카페의 특징은 동물 종류가 각양각색이라는 점이다. 개나 고양이뿐 아니라 부엉이, 고슴도치, 토끼 등으로 특화되어 있다. 도쿄의 번화가 중 하나인 이케부쿠로(池袋)에는 펭귄에게 먹이를 주는 카페가 있을 정도다.\
젊은층과 관광객으로 발 디딜 틈이 없는 하라주쿠(原宿)역 부근, 관광객들에게 인기가 있다는 부엉이 카페를 찾았다. 일본에선 부엉이를 뜻하는 후쿠로(ふくろう) 발음이 복을 뜻하는 후쿠(ふく)와 비슷하고 영화 해리포터에 등장하는 부엉이의 영향으로 부엉이 인기가 많다고 한다. \
온라인에서 예약을 하지 않으면 오래 기다릴 수 있다고 해서 1주일 전 예약을 했다. 평일 오후 2시에 모인 입장객 11명 대부분은 외국인이었다. 1시간 이용하는 데 드는 비용은 1,500엔(약 1만5,000원). 15분은 음료를 마시고, 35분은 부엉이가 있는 데로 들어가 사진을 찍거나 만지고, 이후엔 짐 정리를 하는 식이었다. \
유리문으로 분리된 부엉이 공간은 손님들이 서 있기에 비좁았다. 곧바로 부엉이 11마리에 대한 소개가 시작됐는데 대부분 어디를 만지면 안 된다는 내용이었다. 하지만 너무 빠르게 진행되는 터라 사실 어떤 부엉이가 어떻다는 건지 기억이 나지 않았다.\
일본 도쿄 하라주쿠(原宿)의 한 부엉이 카페에서 손님이 부엉이에게 밥을 주는 체험을 하고 있다. 고은경기자\
일본 도쿄 하라주쿠(原宿)에 있는 부엉이카페에 있는 부엉이. 날아가지 못하도록 발에 줄을 묶어 놓았다. 고은경기자\
다만 부엉이들이 스트레스를 받는 건 그 자리에서도 알 수 있었다. 계속 지지대 위에서 떨어지던 부엉이 한 마리는 결국 커튼 뒤에서 휴식을 취해야 했다. 덩치가 큰 부엉이들이 인기가 많아 아침부터 시달려서인지 주의를 기울여달라고 했다. 그렇다고 만지지 못하게 하거나 사진을 찍지 못하게 할 수는 없었기 때문에 손님들은 큰 부엉이를 손에 앉혔지만 결국 부엉이는 큰 날갯짓을 하며 떨어졌다. 창 밖만 바라보는 부엉이도 있었고 겁에 질려 계속 눈만 껌뻑 거리는 부엉이도 있었다. 날아가지 못하도록 발은 모두 고리에 묶인 채였다. 보통 부엉이 카페를 찾는 사람은 하루에 50~60명. 부엉이들의 ‘고단한’ 하루는 오전 11시부터 오후 8시까지 매 시간 계속됐다. 일본 동물카페 사이트인 애니멀즈닷컴에 따르면 도쿄 근처에 있는 부엉이 카페만 10여곳, 고양이, 토끼, 개 등을 포함하면 50곳이 넘는다.\
일본 내 동물단체와 외신들은 특히 부엉이 카페에 대한 우려를 지속적으로 제기해왔다. 부엉이 카페 운영방식은 야행성 동물인 부엉이의 습성과 정면으로 배치된다. 더욱이 시력과 청력이 좋고 사냥을 하는 본능이 있는데 좁은 공간에 가둔 채 체험에 동원되는 게 심각한 문제라는 것이다. \
기자가 지난 25일 일본 도쿄 하라주쿠(原宿)에 있는 부엉이카페에서 부엉이 한 마리를 손에 올려보고 있다. 고은경 기자\
하지만 부엉이를 비롯해 토끼, 고슴도치 등의 카페는 앞으로도 성행할 것으로 보인다. 일본의 대표적 동물단체인 피스(PEACE)에 따르면 일본 동물애호법에는 동물 카페 개설 조건이 있지만 까다롭지 않아 누구나 마음만 먹으면 열 수 있고, 시설 조건이나 동물 관리방법 등에 대해서도 다루지 않고 있기 때문이다. 또 동물애호법 개정을 앞두고 있지만 일본 환경부는 개와 고양이 이외 다른 동물에 대해서는 기준 마련을 미루고 있다고 했다. \
우리나라도 사정은 다르지 않다. 야생동물을 카페에서 전시하지 못하게 하는 이른바 라쿤카페 금지법이 발의되어 있지만 국회에서 계류 중이다. ‘동물은 괴로운’ 동물카페가 한국에서도 일본에서도 사라지는 날이 오기를 바란다."

# Query 기반 가중치 탐색

In [377]:
queryRepr = defaultdict(int) # 빈도를 갖는

for token in query.split(): 
    for morpheme in ma(token):
        queryRepr[morpheme] += 1
        
queryWeight = defaultdict(float)
maxFreq = max(queryRepr.values())

for token, freq in queryRepr.items():
    if token in TWM.keys():
        tf = maxTF(0.5,freq,maxFreq)
        df = len(TWM[token])
        idf = rawIdf(df,N)
        queryWeight[token] = tf * idf

In [378]:
queryWeight

defaultdict(float,
            {'일본': 0.5837473275379322,
             '도쿄': 0.9552600369393093,
             '(': 0.0,
             ')': 0.0,
             '에': 0.0,
             '는': 0.0,
             '동물': 1.3123738588283629,
             '을': 0.002504440504692245,
             '보': 0.11081867729603334,
             '고': 0.0022808297453447233,
             '만지': 1.1559888533249543,
             'ㄹ': 0.01830626982525732,
             '수': 0.07777204243879954,
             '있': 0.008250783619638425,
             '카페': 1.5592407789034264,
             '가': 0.004580127948738263,
             '많': 0.29109635274502954,
             '다': 0.009332853602541824,
             '.': 0.0,
             '우리나라': 0.7348146199429942,
             '방송': 0.5412909373588878,
             '에서': 0.006898196141009175,
             '연예인': 0.7168923121395064,
             '들': 0.052814956754375954,
             '이': 0.0,
             '를': 0.0,
             '방문': 0.5625179553887468,
             '하': 0.0,
     

In [379]:
def innerProduct(x,y):
    return x * y

In [380]:
from math import sqrt

candidateList = defaultdict(float)

for token, weight in queryWeight.items():
    for fileName, tfidf in TWM[token].items():
        print(" {0} : {1} = {2} * {3}".format(
        token, fileName,weight,tfidf))
        candidateList[fileName] += innerProduct(weight, tfidf)
        
for fileName, sumProduct in candidateList.items():
    candidateList[fileName] /= sqrt(DVL[fileName])

 일본 : ./News/IT과학_0002892931.txt = 0.5837473275379322 * 0.01312926515223723
 일본 : ./News/IT과학_0003520759.txt = 0.5837473275379322 * 0.12754143290744738
 일본 : ./News/정치_0002892931.txt = 0.5837473275379322 * 0.01312926515223723
 일본 : ./News/세계_0002984897.txt = 0.5837473275379322 * 0.031885358226861844
 일본 : ./News/세계_0000718428.txt = 0.5837473275379322 * 0.02665044866722781
 일본 : ./News/생활문화_0003206695.txt = 0.5837473275379322 * 0.02625853030447446
 일본 : ./News/IT과학_0000003645.txt = 0.5837473275379322 * 0.022602279249421056
 일본 : ./News/정치_0004433737.txt = 0.5837473275379322 * 0.030785863115590745
 일본 : ./News/세계_0002984905.txt = 0.5837473275379322 * 0.06377071645372369
 일본 : ./News/세계_0002891514.txt = 0.5837473275379322 * 0.037199584598005486
 일본 : ./News/세계_0000167745.txt = 0.5837473275379322 * 0.04698894896590167
 일본 : ./News/IT과학_0002983549.txt = 0.5837473275379322 * 0.04058136501600599
 일본 : ./News/생활문화_0000025119.txt = 0.5837473275379322 * 0.021256905484574564
 일본 : ./News/세계_00029

 하 : ./News/경제_0010706823.txt = 0.0 * 0.0
 하 : ./News/세계_0002984897.txt = 0.0 * 0.0
 하 : ./News/IT과학_0000484888.txt = 0.0 * 0.0
 하 : ./News/사회_0003434078.txt = 0.0 * 0.0
 하 : ./News/사회_0010695285.txt = 0.0 * 0.0
 하 : ./News/세계_0000718428.txt = 0.0 * 0.0
 하 : ./News/생활문화_0003206695.txt = 0.0 * 0.0
 하 : ./News/IT과학_0000003645.txt = 0.0 * 0.0
 하 : ./News/생활문화_0001268084.txt = 0.0 * 0.0
 하 : ./News/IT과학_0004191439.txt = 0.0 * 0.0
 하 : ./News/세계_0010706108.txt = 0.0 * 0.0
 하 : ./News/세계_0001335075.txt = 0.0 * 0.0
 하 : ./News/세계_0002891496.txt = 0.0 * 0.0
 하 : ./News/경제_0010706985.txt = 0.0 * 0.0
 하 : ./News/IT과학_0002158415.txt = 0.0 * 0.0
 하 : ./News/정치_0002891527.txt = 0.0 * 0.0
 하 : ./News/정치_0002891533.txt = 0.0 * 0.0
 하 : ./News/경제_0004433748.txt = 0.0 * 0.0
 하 : ./News/정치_0000716947.txt = 0.0 * 0.0
 하 : ./News/세계_0010707201.txt = 0.0 * 0.0
 하 : ./News/정치_0004433737.txt = 0.0 * 0.0
 하 : ./News/세계_0003205531.txt = 0.0 * 0.0
 하 : ./News/사회_0004191480.txt = 0.0 * 0.0
 하 : ./News/생활문화_00028

 점 : ./News/IT과학_0002446400.txt = 0.31053759957629934 * 0.015138707979344593
 점 : ./News/생활문화_0010694861.txt = 0.31053759957629934 * 0.005767126849274131
 점 : ./News/정치_0000025834.txt = 0.31053759957629934 * 0.03302990831857002
 점 : ./News/생활문화_0010705943.txt = 0.31053759957629934 * 0.021626725684777987
 점 : ./News/IT과학_0004188529.txt = 0.31053759957629934 * 0.01261558998278716
 점 : ./News/세계_0009114063.txt = 0.31053759957629934 * 0.01892338497418074
 점 : ./News/생활문화_0001512690.txt = 0.31053759957629934 * 0.0550498471976167
 점 : ./News/생활문화_0003204412.txt = 0.31053759957629934 * 0.023592791656121444
 점 : ./News/사회_0010694469.txt = 0.31053759957629934 * 0.07266579830085404
 점 : ./News/IT과학_0010695109.txt = 0.31053759957629934 * 0.013456629314972973
 점 : ./News/사회_0001182881.txt = 0.31053759957629934 * 0.03949228168524676
 점 : ./News/IT과학_0000429582.txt = 0.31053759957629934 * 0.02329031996822245
 점 : ./News/경제_0004110721.txt = 0.31053759957629934 * 0.021626725684777987
 개 : ./News/생활문화_

 으면 : ./News/IT과학_0004188529.txt = 0.349838412856562 * 0.013865546851022273
 으면 : ./News/세계_0009114063.txt = 0.349838412856562 * 0.010399160138266705
 으면 : ./News/생활문화_0010682581.txt = 0.349838412856562 * 0.015126051110206117
 으면 : ./News/IT과학_0004107596.txt = 0.349838412856562 * 0.02376950888746675
 으면 : ./News/경제_0002891478.txt = 0.349838412856562 * 0.017065288432027412
 으면 : ./News/세계_0010694832.txt = 0.349838412856562 * 0.02079832027653341
 으면 : ./News/사회_0010705984.txt = 0.349838412856562 * 0.014468396714110198
 으면 : ./News/정치_0002221071.txt = 0.349838412856562 * 0.010399160138266705
 오래 : ./News/IT과학_0000484888.txt = 1.0753384682092595 * 0.049926428881144194
 기다리 : ./News/IT과학_0002892931.txt = 0.6419548579455382 * 0.09204499801424997
 기다리 : ./News/정치_0002892931.txt = 0.6419548579455382 * 0.09204499801424997
 기다리 : ./News/정치_0002891533.txt = 0.6419548579455382 * 0.035766056371251416
 기다리 : ./News/경제_0010694192.txt = 0.6419548579455382 * 0.027818043844306656
 기다리 : ./News/정치_000419

 줄 : ./News/생활문화_0000025119.txt = 0.3972772914226344 * 0.01844501710176517
 줄 : ./News/경제_0002891478.txt = 0.3972772914226344 * 0.03972772914226344
 줄 : ./News/세계_0010694832.txt = 0.3972772914226344 * 0.024209084946066786
 줄 : ./News/사회_0001182881.txt = 0.3972772914226344 * 0.05052330771353068
 묶 : ./News/세계_0000718428.txt = 1.0753384682092595 * 0.03129716437325457
 놓 : ./News/경제_0009113386.txt = 0.44433902134860637 * 0.018051272742287135
 놓 : ./News/생활문화_0004433728.txt = 0.44433902134860637 * 0.0346584436651913
 놓 : ./News/IT과학_0002892931.txt = 0.44433902134860637 * 0.02548414975381713
 놓 : ./News/정치_0002892931.txt = 0.44433902134860637 * 0.02548414975381713
 놓 : ./News/경제_0009113330.txt = 0.44433902134860637 * 0.015753838029632407
 놓 : ./News/생활문화_0003206695.txt = 0.44433902134860637 * 0.02548414975381713
 놓 : ./News/IT과학_0000003645.txt = 0.44433902134860637 * 0.02193572383872867
 놓 : ./News/정치_0004433737.txt = 0.44433902134860637 * 0.02987796867688905
 놓 : ./News/정치_0010695232.txt =

 와 : ./News/IT과학_0000003645.txt = 0.04641927552681659 * 0.004471384366893108
 와 : ./News/생활문화_0001268084.txt = 0.04641927552681659 * 0.006307845803295634
 와 : ./News/세계_0010706108.txt = 0.04641927552681659 * 0.008410461071060846
 와 : ./News/세계_0002891496.txt = 0.04641927552681659 * 0.002848704556327061
 와 : ./News/경제_0010706985.txt = 0.04641927552681659 * 0.006090333879044061
 와 : ./News/IT과학_0002158415.txt = 0.04641927552681659 * 0.01962440916580864
 와 : ./News/정치_0002891527.txt = 0.04641927552681659 * 0.009057419614988604
 와 : ./News/정치_0002891533.txt = 0.04641927552681659 * 0.002523138321318254
 와 : ./News/경제_0004433748.txt = 0.04641927552681659 * 0.008546113668981182
 와 : ./News/세계_0010707201.txt = 0.04641927552681659 * 0.006307845803295634
 와 : ./News/정치_0004433737.txt = 0.04641927552681659 * 0.009135500818566091
 와 : ./News/세계_0003205531.txt = 0.04641927552681659 * 0.008028167386012626
 와 : ./News/사회_0004191480.txt = 0.04641927552681659 * 0.0033965323556207267
 와 : ./News/생활문화_00

In [389]:
# 생활문화만 K가 7이 넘어야 잘 분류함 

from nltk.tokenize import sent_tokenize
from collections import Counter
K=7

resultList = sorted(candidateList.items(), key = lambda x:x[1], reverse=True)
catList = list()

for i,(fileName, similarity) in enumerate(resultList):
    if i < K:
        cat = fileName.split('/')[2]
        category = cat.split('_')[0]
        catList.append(category)
        
        result = Counter(catList)
print(result.most_common()[0])
        

('생활문화', 3)


# chromedriver

In [565]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/Users/charming/Python/3. Koipa AI Learning/NLP/chromedriver')
driver.get('https://nid.naver.com/nidlogin.login?mode=form')

In [571]:
driver.get('https://mail.naver.com/?n=1552462013012&v=f#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%225%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D')

In [586]:
tagList = list()
for tag in driver.find_elements_by_css_selector('strong.mail_title'):
    tagList.append(tag.text)

In [587]:
tagList

['(제목없음)',
 '(제목없음)',
 '(광고) 로또1등담첨 예상번호 무료로 받아가세요~',
 '(광고) 성공을 위한 1인무점포 창업과 다양한 창업아이템 설명회 안내',
 '(광고) 더 이상 채.무. 로 고통받지 마세요',
 '(광고) 이번 Apple 스페셜 이벤트에서 발표된 놀라운 소식을 살펴보세요.',
 '매일매일 이벤트 후불실시']

In [588]:
import requests
session = requests.Session()

In [589]:
for c in driver.get_cookies():
    print(c['name'], c['value'])
    session.cookies.set(c['name'], c['value'])
    # set으로 다 밀어넣으면됨

NMUSER Y9YmFAEwaqbsFAKrKA2w+Hi9p69CWov9KxUsKxE9FAUXFqvdKqn9KxUmaqgsaqRJaw/wFxRpad/syqvstoRpadUstonstoRzaqROW9e7EoRpadUsawlGW430DVdq74lR74lC+4kZ74FTWLm/axgmar05pzk/7xKRbrkoWrlvMBil74lR74lC+4kZ74FTWLm/axgmam==
NNB RG4EQLZMM2SFY
NID_JKL HgeG0711CfMqBp42TfhWF7cEKTky/HU0TEzkNOEkKCo=
nid_inf 1831237061
NID_SES AAABlWppGZjlaH7+pRs9gOMNZjSXz6CXNE9bkZFfZrHxaznVYRvGrDXpuvHpdNxiP3X5z7rXEhnWFxwaxJWWELmlVbounFyekOlV2fL+ToqrGFimoHN9cGXUV9D6LMf575DBEsqMJsM6PLRVBPNb8GjK3H50blJwXVdNNBZP00yfwSJGZ+lZjYdwtdHISjbPWYGsvsh+96snF9M+8vv1oaaQoeCt80YE0djrd4cYb3He5p1haPhMcNQfGWBvicz6v9zjRsfNZcyEFKfm7LkkHyI7BD98Dt+7XL6SxoXOqCk/vTpHXI/7qciEAongjRawR9WgNZvnlTg0FjeHTqOIlf5smlqkV8Ws0SRYQ2nx0FOq12WKYIKYqDXKadd8jk5OfwVB8/Ch6zdcNgoFqjMOVR1yH0s2rKtErQWHQzQdpjBqfnkaVCk0HwMCEx3YbY4/rA3tozAsj+xhylTblICzkj0NKS+3l7wrFDCcaujWUiMToCIPnV1xAXTVXV3DK6gBviJ0ibw9zxGhRk23wPqJQjsziFniAQxVCZbwruDwZroupKO0
NID_AUT aF8pWmMYt1z7X1zQZoH0YPCmOKKEDk8U/rkbnD9idqO55fsNOgm8LCp7/R5H/QgL
npic SBlKFg2n6N1KqKmWNB4f+E3omgk1w/ADCsSLmaJSBIUCtwy80MA8gkb

In [590]:
session.cookies.get_dict()

{'NID_AUT': 'aF8pWmMYt1z7X1zQZoH0YPCmOKKEDk8U/rkbnD9idqO55fsNOgm8LCp7/R5H/QgL',
 'NID_JKL': 'HgeG0711CfMqBp42TfhWF7cEKTky/HU0TEzkNOEkKCo=',
 'NID_SES': 'AAABlWppGZjlaH7+pRs9gOMNZjSXz6CXNE9bkZFfZrHxaznVYRvGrDXpuvHpdNxiP3X5z7rXEhnWFxwaxJWWELmlVbounFyekOlV2fL+ToqrGFimoHN9cGXUV9D6LMf575DBEsqMJsM6PLRVBPNb8GjK3H50blJwXVdNNBZP00yfwSJGZ+lZjYdwtdHISjbPWYGsvsh+96snF9M+8vv1oaaQoeCt80YE0djrd4cYb3He5p1haPhMcNQfGWBvicz6v9zjRsfNZcyEFKfm7LkkHyI7BD98Dt+7XL6SxoXOqCk/vTpHXI/7qciEAongjRawR9WgNZvnlTg0FjeHTqOIlf5smlqkV8Ws0SRYQ2nx0FOq12WKYIKYqDXKadd8jk5OfwVB8/Ch6zdcNgoFqjMOVR1yH0s2rKtErQWHQzQdpjBqfnkaVCk0HwMCEx3YbY4/rA3tozAsj+xhylTblICzkj0NKS+3l7wrFDCcaujWUiMToCIPnV1xAXTVXV3DK6gBviJ0ibw9zxGhRk23wPqJQjsziFniAQxVCZbwruDwZroupKO0',
 'NMUSER': 'Y9YmFAEwaqbsFAKrKA2w+Hi9p69CWov9KxUsKxE9FAUXFqvdKqn9KxUmaqgsaqRJaw/wFxRpad/syqvstoRpadUstonstoRzaqROW9e7EoRpadUsawlGW430DVdq74lR74lC+4kZ74FTWLm/axgmar05pzk/7xKRbrkoWrlvMBil74lR74lC+4kZ74FTWLm/axgmam==',
 'NNB': 'RG4EQLZMM2SFY',
 'nid_inf': '1831237061',
 'npic': 'SBlKFg2n

In [591]:
tagList = list()

for tag in driver.find_elements_by_css_selector('strong.mail_title'):
    tagList.append(tag)

In [592]:
print(len(tagList))

7


In [593]:
# 본문 내용 찾기
# 메일 들어가서 실행
driver.find_element_by_id('readFrame').text

''

In [594]:
# delay
import time
import random
# time.sleep(random.randint(1,2))

In [597]:
tagList = list()
for tag in driver.find_elements_by_css_selector('strong.mail_title'):
    tagList.append(tag.text)
    
tagList

['(제목없음)',
 '(제목없음)',
 '(광고) 로또1등담첨 예상번호 무료로 받아가세요~',
 '(광고) 성공을 위한 1인무점포 창업과 다양한 창업아이템 설명회 안내',
 '(광고) 더 이상 채.무. 로 고통받지 마세요',
 '(광고) 이번 Apple 스페셜 이벤트에서 발표된 놀라운 소식을 살펴보세요.',
 '매일매일 이벤트 후불실시']

In [599]:
# Spam mail Crawling
# 스팸 메일 수집하기

spamList = list()
for i in range(len(tagList)):
    driver.get('https://mail.naver.com/?n=1552462013012&v=f#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%225%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D')    
    time.sleep(random.randint(1,3))
    driver.find_elements_by_css_selector('strong.mail_title')[i].click()
    time.sleep(random.randint(1,5))
    text = driver.find_element_by_id('readFrame').text
    spamList.append([tagList[i],text])  # 제목부분 수정
    
    driver.get('https://mail.naver.com/?n=1552462013012&v=f#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%225%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D')    
    
print(spamList)

[['(제목없음)', '  ◎상단이미지링크표시◎바로가기◎'], ['(제목없음)', '궁금하신점 있으시면 매니저와 상담하세요\n천 연 발 기 제 대 방 출'], ['(광고) 로또1등담첨 예상번호 무료로 받아가세요~', '본 메일은 (주)엔비즈에서 시행하는 이벤트 참여자중 메일 받기를 동의하신 분들께 발송되는 메일 입니다.\n수신을 더이상 원하지 않는 분들께서는 수신거부를 클릭해 주시기 바랍니다.\nTo unsubscribe this e-mail, click here\n\n소재지 : 인천광역시 연수구 인천타워대로323 C동 906호 (주)엔비즈\n사업자등록번호 : 621-88-00616 T.1644-7410 대표메일 : hyh3513@daum.net\nCopyright ⓒ NBIZ Co., Ltd. All rights reserved.'], ['(광고) 성공을 위한 1인무점포 창업과 다양한 창업아이템 설명회 안내', '(광고) 성공을 위한 1인무점포 창업과 다양한 창업아이템 설명회 안내\n\n1. 수도권 최고의 놀이동산 내 푸드코트, 권리금, 인테리어 비용 없는 소자본창업 사업설명회\n날 짜 : 3 월 29 일 금요일 오후 2 시 ~ [자세히 보기]\n\n2. 세계 NO 1 온라인쇼핑몰 아마존과 급성장중인 해외직구쇼핑몰 동시 창업과정 무료세미나.\n(서울, 부산, 광주에서도 동시에 같은 날에 동시에 진행합니다.)\n서울본사 / 부산지사 / 광주지사 : 4 월 2일(화요일) 오후 2 시 - 5 시 (3시간)\n서울본사 / 부산지사 / 광주지사 : 4 월 9일(화요일) 오후 2 시 - 5 시 (3시간)  [자세히 보기]\n\n3. 열린창업신문, 성공창업을 만들어가는 창업컨설팅 사업을 함께 할 전국 지사모집 사업설명회\n날 짜 : 4 월 5 일 금요일 오후 2시 ~ [자세히 보기]\n\n4. “유튜브 창업과정”, “SNS 1인대행사 창업과정” & “SNS마케팅 전문가과정” 합동세미나\n1억 연봉의 노하우가 합동세미나에서 공개 됩니다.\n날 짜 : 4월 11일 일요일 오

In [600]:
# 개수, 내용 확인
print(len(spamList)),spamList[2]

7


(None,
 ['(광고) 로또1등담첨 예상번호 무료로 받아가세요~',
  '본 메일은 (주)엔비즈에서 시행하는 이벤트 참여자중 메일 받기를 동의하신 분들께 발송되는 메일 입니다.\n수신을 더이상 원하지 않는 분들께서는 수신거부를 클릭해 주시기 바랍니다.\nTo unsubscribe this e-mail, click here\n\n소재지 : 인천광역시 연수구 인천타워대로323 C동 906호 (주)엔비즈\n사업자등록번호 : 621-88-00616 T.1644-7410 대표메일 : hyh3513@daum.net\nCopyright ⓒ NBIZ Co., Ltd. All rights reserved.'])

In [602]:
spamList[2][0] # 제목
spamList[2][1] # 내용

'본 메일은 (주)엔비즈에서 시행하는 이벤트 참여자중 메일 받기를 동의하신 분들께 발송되는 메일 입니다.\n수신을 더이상 원하지 않는 분들께서는 수신거부를 클릭해 주시기 바랍니다.\nTo unsubscribe this e-mail, click here\n\n소재지 : 인천광역시 연수구 인천타워대로323 C동 906호 (주)엔비즈\n사업자등록번호 : 621-88-00616 T.1644-7410 대표메일 : hyh3513@daum.net\nCopyright ⓒ NBIZ Co., Ltd. All rights reserved.'

In [609]:
tagList = list()
for tag in driver.find_elements_by_css_selector('strong.mail_title'):
    tagList.append(tag.text)
    
tagList

['Movie & Beer의 주간 Facebook 페이지 업데이트',
 '(광고) 품격을 높이는 프리미엄 브랜드 쇼핑 ▶ 카드사 무제한 7%할인',
 '이차민 님! 새로운 1촌 Farrukh 님과 대화해 보세요.',
 'Competition Launch: Jigsaw Unintended Bias in Toxicity Classification',
 "[매일경제] 靑민정수석, '독이 든 성배' 쥔 수석 중의 수석",
 '(광고) 일본으로 떠나는 휴양, 내가 바라던 바다!',
 '(광고) 쟈뎅 까페리얼 토피라떼 230ml 20팩 7,900원',
 'Sci-fi tracks that’ll take your work to the next level',
 '[안내] NAVER 입사지원서 접수 완료',
 '이차민 님의 프로필 조회수가 올라가고 있습니다.',
 '이차민 님! 새로운 1촌 Jun duk(준덕) 님과 대화해 보세요.',
 '(광고) [G마켓 G9] G9에서 30% 할인을 준비했어요!',
 '(광고) 이차민님, 잘지내셨나요?',
 "(광고)[온오프믹스 뉴스] '나'를 아끼는 현명한 습관, 4월 첫번째 모임 소식!!",
 '이차민 님에게 추천해 드리는 1촌']

In [610]:
# normal mail Crawling
# 정상 메일 수집하기

hamList = list()
for i in range(len(spamList)):
    driver.get('https://mail.naver.com/?n=1552462013012&v=f#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%220%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D')    
    time.sleep(random.randint(1,3))
    driver.find_elements_by_css_selector('strong.mail_title')[i].click()
    time.sleep(random.randint(1,5))
    text = driver.find_element_by_id('readFrame').text
    hamList.append([tagList[i],text])
    
    driver.get('https://mail.naver.com/?n=1552462013012&v=f#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%220%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D')    
# print(hamList)

[['Movie & Beer의 주간 Facebook 페이지 업데이트', 'See insights about your Page between 3월 27일 and 4월 2일.\nMovie & Beer의 페이지 인사이트 요약\n3월 6일~4월 2일\n  2\n0\n0\n게시물 도달\n게시물 참여\n새로운 페이지 좋아요\n-50%\n  트렌드는 현재와 지난 28일간을 비교하여 산출됩니다.\n  인사이트 더 보기\n내 게시물\n최근 28일 동안 게시된 게시물이 없습니다. 사진 또는 동영상이 포함된 게시물을 만들어 회원님의 페이지를 좋아하는 사람들의 참여를 유도하세요.\n게시물 만들기\n사진을 게시하여 참여를 유도해보세요\n회원님의 페이지와 유사한 페이지에서 사진을 게시하면 사람들의 참여가 더 많아지는 경향이 있습니다.\n사진 게시물 만들기\n  trueman9512@naver.com 주소로 전송된 메시지입니다. 앞으로 Facebook으로부터 이와 같은 이메일을 받지 않으려면 그만 받기를 누르세요.\nFacebook, Inc., Attention: Community Support, 1 Facebook Way, Menlo Park, CA 94025\n계정을 안전하게 유지하기 위해 이 이메일을 전달하지 마세요. 더 알아보기.\n '], ['(광고) 품격을 높이는 프리미엄 브랜드 쇼핑 ▶ 카드사 무제한 7%할인', '이미지가 보이지 않거나 클릭이 되지 않으실 경우 여기를 클릭해주세요.\nG9는 통신판매중개자이며 통신판매의 당사자가 아닙니다. 따라서 G9는 상품, 거래정보 및 거래에 대하여 책임을 지지 않습니다.\n본 메일은 2019년 4월 2일 기준 회원님의 광고이메일 수신 동의 여부를 확인한 결과 수신을\n동의하셨기에 발송되었습니다. 수신을 원치않으시면 여기를 클릭하여 주시기 바랍니다.\nIf you don’t want to receive this email anymore, Click here.\n본 메일은 발신전용 메일이므로 문의사항은 g9_help@g9.co.kr로 문의

In [611]:
# 개수, 내용 확인
print(len(hamList)),hamList[3]

7


(None,
 ['Competition Launch: Jigsaw Unintended Bias in Toxicity Classification',
  " \nHi @leechamin,\nWe're excited to let you know about a new competition by Jigsaw! Can you help detect toxic comments ― and minimize unintended model bias? That's your challenge in this year's edition. Click the button at the bottom of this email to get started.\nAbout the Competition\nIn this competition, you're challenged to build a model that recognizes toxicity and minimizes unintended bias with respect to mentions of identities. You'll be using a dataset labeled for identity mentions and optimizing a metric designed to measure unintended bias. Develop strategies to reduce unintended bias in machine learning models, and you'll help the Conversation AI team, and the entire industry, build models that work well for a wide range of conversations.\nAbout the Host\nThe Conversation AI team, a research initiative founded by Jigsaw and Google (both part of Alphabet), builds technology to protect voices i

In [612]:
hamList[2][0] # 제목
# hamList[2][1] # 내용

'이차민 님! 새로운 1촌 Farrukh 님과 대화해 보세요.'

# 다음에서도 스팸/햄 메일 가져오기

In [651]:
driver.get('http://mail2.daum.net/hanmailex/Top.daum?checkGroot=false')

In [652]:
driver.find_elements_by_id('id-%EC%8A%A4%ED%8C%B8%ED%8E%B8%EC%A7%80%ED%95%A8-maillist')

[]

In [653]:
driver.get('http://mail2.daum.net/hanmailex/Top.daum?checkGroot=false')

In [655]:
# spam 제목 부르기
daumSpamList = list()

for i in range(16,30):
    text = driver.find_element_by_id( str(i) +'-mailitem-subject-text').text
    daumSpamList.append(text)
    

In [656]:
daumSpamList

['🕑 라이브 9 Casino 대박 빅 3 이벤트.❕ ＜ 오링, 콤프, 충전 ＞ 신규 회원 모집중️ 🕑',
 '💕 비밀포장+빠른배송 - 비야그라💊씨얄리스💊여성최음제💊남성발기제등 - 하나사면 하나더!!! 💕',
 '🎣 즉 시 오 만 바 로 충 전 🎣 역 대 환 수 율 🎣모 바 일 & P C 완 벽 호 환 !! ...',
 '☆짜릿한 타격감과 『96% 환수율』 『무료머니 5만으로 먼저 체험 하세요!』☆',
 '📧📧 200% 정품 비아그라 시알리스 / 온라인 최저가 판매 / 정품아닐시 200% 보상',
 '🍄 🍄 하나가격에 두개를 특가 ▶ 파격세일 효과100% 사정 지연 ♡ 비밀포장 _ 안심배송',
 'wxwdeoo owh i',
 '💜 💜 여성최음제 특가 이벤트 💜 9만 9천원 파격특가 💜 특별 사은품 증정',
 '제목없음',
 '🎰 종합 게임매장👍 < 신천지 오션파라다이스, 야마토, 바다이야기, 스위피 > 멀티가능 🎰',
 '제목없음',
 '제목없음',
 '📧📧 200% 정품 비아그라 시알리스 / 온라인 최저가 판매 / 정품아닐시 200% 보상',
 '☆짜릿한 타격감과 『96% 환수율』 『무료머니 5만으로 먼저 체험 하세요!』☆']

In [649]:
# ham 메일 제목 부르기
daumHamList = list()

for i in range(47,61):
    text = driver.find_element_by_id( str(i) +'-mailitem-subject-text').text
    daumHamList.append(text)

In [650]:
daumHamList

['(광고) 쟈뎅 까페리얼 토피라떼 230ml 20팩 7,900원',
 '(광고) 2019 4월의 HOT한 급등종목 3가지를 공개합니다.',
 '(광고) [G마켓 G9] G9에서 30% 할인을 준비했어요!',
 'TOP point - 이차민님의 소멸 예정 포인트를 알려드립니다 (50994723)',
 '(광고) 샤오미 알고 사는게 이득! ▶ 신기한 샤오미 제품부터 정보까지!',
 '(광고) [국비지원 무료교육] 신재생에너지 발전설비기사 자격시험 안내 및 무료자료 제공',
 '(광고) 봄맞이 취향저격 e쿠폰 특가 만나보세요!',
 '(광고) 국비지원 주택관리사 // 자격증 쉽게 취득하는 방법 공개!! / 놓치면 늦는다!',
 '(광고) 2019 4월의 HOT한 급등종목 3가지를 공개합니다.',
 '(광고) 안 할 이유 없죠? TOP 11 소호몰 스마일페이 결제 시 최대 12.5% 적립',
 '[매거진뉴스] 숨 쉬고 사는 문제',
 '(광고) [금강제화] 품격이 남다른 남성 캐주얼 슈즈 모음 40,000원부터~',
 '(광고) 운동복과 일상복 그사이! 스포츠웨어 애슬레저룩!',
 '(광고) 쿠폰 혜택 가득! 더 알뜰한 살림장만 스마일배송으로']

In [642]:
str(1) +'-mailitem-subject-text'

'1-mailitem-subject-text'